In [1]:
import os
import pandas as pd
from utils import find_csv

In [4]:
f = "data/news_title_sentimentscore.csv"

df = pd.read_csv(f)
df.rename(columns = {"News_Headline": "Date"}, inplace = True)
df.head()

,Date,Positive,Negative,Neutral,Prediction
0,2020-12-31,0.000794,0.998949,0.000257,Negative
1,2020-12-31,0.000421,0.000207,0.999372,Neutral
2,2020-12-29,0.999180,0.000609,0.000211,Positive
3,2020-12-29,0.999180,0.000609,0.000211,Positive
4,2020-12-28,0.999180,0.000609,0.000211,Positive


In [5]:
df['Date'] = pd.to_datetime(df['Date'])

,Date,Positive,Negative,Neutral,Prediction
0,2020-12-31,0.000794,0.998949,0.000257,Negative
1,2020-12-31,0.000421,0.000207,0.999372,Neutral
2,2020-12-29,0.999180,0.000609,0.000211,Positive
3,2020-12-29,0.999180,0.000609,0.000211,Positive
4,2020-12-28,0.999180,0.000609,0.000211,Positive


In [3]:
path = "/Users/shenchingfeng/GitHub/GDSC-ai-stock/News_History"

df = find_csv(path)
# df.drop(columns = ["Unnamed: 0"], inplace = True)
df.sort_values("Date", ascending = False, inplace = True)
df.drop_duplicates(inplace = True, keep = "last")

In [4]:
df.head(40)

,Date,News Title
41,2024-04-29,Musk makes surprise visit to Beijing as Tesla’...
40,2024-04-28,Elon Musk visits China as Tesla seeks self-dri...
42,2024-04-28,How working for Big Tech lost ‘dream job’ status
44,2024-04-27,Nvidia jumps more than 15% this week. A key re...
43,2024-04-27,Buy the dip on tech stocks like Apple and Nvid...
45,2024-04-27,Federal regulator finds Tesla Autopilot has ‘c...
48,2024-04-26,Chinese EV start-ups Nio and Xpeng turn to the...
52,2024-04-26,Tesla driver in Seattle-area crash that killed...
51,2024-04-26,India ‘confident’ new EV policy measures will ...
50,2024-04-26,Former Tesla SVP Drew Baglino is selling $181....


In [5]:
df.sort_values("Date", ascending = False).to_csv(f"{path}/News_Titile.csv", index = False)